## Many-to-many relationships in GeMS]

As of version 1.6.12 of the ArcMap and version 2.5.3 of the ArcGIS Pro GeMS Toolbox, you can concatenate primary `DataSource_IDs` in a `DataSourceID` field to relate multiple data sources to a single row in a standalone table or feature class. Separate the IDs by a pipe character, like so:

`DAS1 | DAS3 | DAS12`

The Validate Database tool will parse the IDs and look each one up in `DataSources`.

To reproduce some of the functionality built in to ArcGIS to edit and query many-to-many relationships this notebook has some code snippets that you can run in ArcGIS Pro alongside GeMS layers. It would probably be useful to turn these snippets into geoprocessing tools or button-activated commands at some point, but in the interest of expediency, they are available here.

### Building a `DataSourceID` string from selected data sources

Before running the cell below:

1. Change the value of `foreign_table` to the name of the feature classs or standalone table in the currently active map to which you want to write concatenated `DataSource_IDs`.
2. Open 'DataSources'
3. Select one or more features or rows from your `foreign_table` layer
4. Set `overwrite` to `True` for overwriting existing values or `False` for appending to existing values.
4. Put your cursor inside the code cell below and press shift-enter.

In [27]:
# SET THE NEXT FIVE VARIABLES
#---------------------------------------------
# name of the sources table, usually DataSources, but
# it doesn't have to be
primary_table = "DataSources"

# name of the primary field in that table
primary_field = "DataSources_ID"

# name of the GeMS feature layer or table that will be updated
foreign_table = "DescriptionOfMapUnits"

# name of the SourceID field that will be updated
foreign_field = "DataSourceID"

# value of the overwrite variable
overwrite = False

#---------------------------------------------
# get a reference to the active map
aprx = arcpy.mp.ArcGISProject('CURRENT')
active_map = aprx.activeMap

#build a list of the currently selected DataSources_IDs
with arcpy.da.SearchCursor(primary_table, primary_field) as rows:
    ids = [row[0].strip() for row in rows]

# write the concatenated data sources string to the field
# or append it, depending on the value of overwrite
with arcpy.da.UpdateCursor(foreign_table, foreign_field) as rows:
    for row in rows:
        if overwrite:
            new_ids = ids
        else:
            old_ids = [n.strip() for n in row[0].split("|")]
            old_ids.extend(ids)
            new_ids = old_ids
                
        row[0] = ' | '.join(sorted(new_ids))
        rows.updateRow(row)

### Selecting foreign table rows based on DataSource

One way to select rows from a table based on the SourceIDs in a particular field is to use the built-in Select By Attributes dialog and build a select statement in SQL, eg.,

`DataSourceID LIKE ('%DAS8%') AND DataSourceID LIKE ('%DAS5%')`

Concatenate LIKE operators for each value you want to search for.

With the cell block below, you can also select rows from the `DataSources` table and when you execute the code the features or rows from the foreign table will be selected.

Choices for `select_method` are

* NEW —Creates a new feature selection from the oidList.
* DIFFERENCE —Selects the features that are not in the current selection but are in the oidList.
* INTERSECT —Selects the features that are in the current selection and the oidList.
* SYMDIFFERENCE —Selects the features that are in the current selection or the oidList but not both.
* UNION —Selects all the features in both the current selection and those in the oidList.

See documentation for the `setSelectionSet` method of the [layer object](https://pro.arcgis.com/en/pro-app/2.8/arcpy/mapping/layer-class.htm)

*btw, the method I use below to select rows is apparently much faster than using the `Select By Attributes` tool.*

In [55]:
# SET THE NEXT FIVE VARIABLES
#---------------------------------------------
# name of the sources table, usually DataSources, but
# it doesn't have to be
primary_table = "DataSources"

# name of the primary field in that table
primary_field = "DataSources_ID"

# name of the GeMS feature layer or table to select from
foreign_table = 'DescriptionOfMapUnits'

# name of the SourceID field in that table
foreign_field = "DescriptionSourceID"

# selection type variable
selection_type = 'NEW'
#---------------------------------------------

# build a list of the currently selected DataSources_IDs
with arcpy.da.SearchCursor(primary_table, primary_field) as rows:
    src_ids = [row[0].strip() for row in rows]

# build a list of OBJECTIDs in the foreign_table where
# the source_ids are found in the source_fld
fields = ['OBJECTID', foreign_field]
oids = []
with arcpy.da.SearchCursor(foreign_table, fields) as rows:
    for row in rows:
        if row[1]:
            ids = [n.strip() for n in row[1].split("|")]
            check =  all(item in ids for item in src_ids)
            if check:
                oids.append(row[0])

# get a pointer to the feature layer or table
if active_map.listLayers(foreign_table):
    lyr = active_map.listLayers(foreign_table)[0]
else:
    lyr = active_map.listTables(foreign_table)[0]

# and apply the selection set ids
lyr.setSelectionSet(oids, selection_type)

### Display related rows in a pop-up dialog

Because we are not using a built-in ArcGIS relationship class, we won't see related rows when querying a feature using the Explore tool, but we can use Arcade to customize the pop-up. There is more [here](https://pro.arcgis.com/en/pro-app/latest/help/mapping/navigation/configure-pop-ups.htm) about configuring pop-ups but below is an expression you can use when you do that.

Arcade code is meaningless within a Jupyter Notebook. It is only in a Python cell for formatting, but you can't run it from here. Change the variables as necessary and save the snippet as a text file with a `.lxp` extension and you can import the expression when necessary to avoid copy/paste.

In [ ]:
var arr = split($feature.DataSourceID, "|");
var tbl = FeatureSetByName($map,"DataSources");
var pop_text = []
if (Count(arr) > 0) {
    for (var i in arr) {
        var ds = Trim(arr[i]);
        var sql = "DataSources_ID = '" + ds + "'";
        var related_data = Filter(tbl, sql);
        var cnt = Count(related_data);
        if (cnt > 0) {
            // loop through related records
            for (var row in related_data) {
                Push(pop_text, ds + ": " + row.Source)
     }}}};
return Concatenate(pop_text, TextFormatting.NewLine)

### Create m:m relationship class from concatenated primary ids

If you try the method in this notebook but decide instead to use an ArcGIS many-to-many relationship class, you don't have to start from scratch. Use the code below to build an intermediate, or junction, table from a table with concatenated ids that can be used to create a many-to-many relationship.

In [ ]:
import os

# SET THE NEXT FIVE VARIABLES
#---------------------------------------------
# name of the sources table, usually DataSources, but
# it doesn't have to be
primary_table = "DataSources"

# name of the primary field in that table
primary_field = "DataSources_ID"

# name of the GeMS feature layer or table to select from
foreign_table = "DescriptionOfMapUnits"

# name of the <table>_ID field in the foreign table
id_field = "DescriptionOfMapUnits_ID"

# name of the SourceID field in that table
foreign_field = "DescriptionSourceID"
#---------------------------------------------

# create an empty intermediate table in memory.
# this will appear in your Contents pane. If you want to save the table to 
# your gdb, use the right-click Data > Export Table command
table_name = f"{foreign_table}_{primary_table}"
int_table = arcpy.CreateTable_management('in_memory', table_name)
arcpy.AddField_management(int_table, id_field, "TEXT")
arcpy.AddField_management(int_table, primary_field, "TEXT")

# fill the table
insert = arcpy.da.InsertCursor(int_table, [id_field, primary_field])
fields = [id_field, foreign_field]
with arcpy.da.SearchCursor(foreign_table, fields) as rows:
    for row in rows:
        if row[1]:
            ids = [n.strip() for n in row[1].split("|")]
            for i in ids:
                insert.insertRow((row[0], i))
del insert 

# get a reference to the active map
aprx = arcpy.mp.ArcGISProject('CURRENT')
active_map = aprx.activeMap

# get a pointer to the foreign feature layer or table
if active_map.listLayers(foreign_table):
    lyr = active_map.listLayers(foreign_table)[0]
else:
    lyr = active_map.listTables(foreign_table)[0]
wksp = os.path.dirname(lyr.dataSource)

# create the relationship class
# lots of tedious parameters!
origin_table = primary_table
destination_table = foreign_table
out_relationship_class = os.path.join(wksp, table_name)
relationship_type = "SIMPLE"
forward_label = foreign_table
backward_label = primary_table
message_direction = "NONE"
cardinality = "MANY_TO_MANY"
relationship_table = int_table
attribute_fields = f"{id_field};{primary_field}"
origin_primary_key = primary_field
origin_foreign_key = primary_field
destination_primary_key = id_field
destination_foreign_key = id_field

arcpy.management.TableToRelationshipClass(origin_table, destination_table, out_relationship_class, relationship_type, forward_label, backward_label, message_direction, cardinality, relationship_table, attribute_fields, origin_primary_key, origin_foreign_key, destination_primary_key, destination_foreign_key)

arcpy.management.Delete(int_table)

### Convert a relationship to concatenated IDs

If you have a many-to-many relationship but want to use the method outlined here of concatenated ids, use the code below. First, add the relationship class you want converted to the map. It will show up as a table.

In [ ]:
# SET THE NEXT VARIABLE
#---------------------------------------------
# full path of the relationship class
rel_table = r"C:\_AAA\gems\examples\Challis\Dillon\Dillon.gdb\DescriptionOfMapUnits_DataSources"
#---------------------------------------------

# get a reference to the active map
aprx = arcpy.mp.ArcGISProject('CURRENT')
active_map = aprx.activeMap

# make a table view of the relationship table
ddTab = arcpy.mp.Table(rel_table)
arcpy.management.MakeTableView(rel_table, 'm2m_view')